1. The textbook mentions namespaces and encapsulation as two of the features that module systems can provide. How does the OCaml module system provide these features?

Modules allow you to hide values by not exposing them in the signature, i.e. in the following `g` and `h` are essentially private, since they arent exposed.

```ocaml
module type A = sig
  type t
  val f : t -> t -> t
end

module MyA : a = struct
    type t = Num of int
    let g = ...
    let h = ...
    let f x = ...
end
```

Namespaces are pretty obvious

```ocaml
module type A = sig
    type t = int
    val f = ...
end

module type B = sig
    type t = int
    val f = ...
end
```

Both of these have a val f, but they're scoped to the parent module, e.g. `A.f` and `B.f`


2. In the stack and queue implementations in the textbook, there are two implementations: one in which some operations raise exceptions, and another in which some operations return options. What are the trade-offs between those implementations?

That is obvious

3. Do the exercises named complex synonym, complex encapsulation, and fraction.

In [4]:
(* Exercise: complex synonym [✭]

Here is a signature for complex numbers, which have a real and imaginary component:

module type ComplexSig = sig
  val zero : float * float
  val add : float * float -> float * float -> float * float
end

Improve that code by adding type t = float * float to the signature. Show how the 
signature can be written more tersely because of the type synonym. *)


module type ComplexSig = sig
  type t = float * float
  val zero : t
  val add : t -> t -> t
end

module type ComplexSig =
  sig type t = float * float val zero : t val add : t -> t -> t end


In [8]:
(* Exercise: complex encapsulation [✭✭]

Here is a structure that matches the signature from the previous exercise:

module Complex : ComplexSig = struct
  type t = float * float
  let zero = (0., 0.)
  let add (r1,i1) (r2,i2) = r1 +. r2, i1 +. i2
end

Investigate what happens if you make the following changes (each independently), 
and explain why any errors arise:

    remove zero from the structure
    remove add from the signature
    change zero in the structure to let zero = 0, 0
 *)
 
(* 1. 
Error: Signature mismatch:
       Modules do not match:
         sig
           type t = float * float
           val add : float * float -> float * float -> float * float
         end
       is not included in
         ComplexSig
       The value `zero' is required but not provided
       File "[4]", line 16, characters 2-14: Expected declaration
       
        *)
        
(* 2. 
Error: Signature mismatch:
       Modules do not match:
         sig type t = float * float val zero : float * float end
       is not included in
         ComplexSig
       The value `add' is required but not provided
       File "[4]", line 17, characters 2-23: Expected declaration
       *)
        
(* 3. 
Error: Signature mismatch:
       ...
       Values do not match:
         val zero : int * int
       is not included in
         val zero : t
       File "[4]", line 16, characters 2-14: Expected declaration
       File "[7]", line 45, characters 6-10: Actual declaration
     *)

In [137]:
(* Exercise: fraction [✭✭✭]

Write a module that implements the Fraction module type below:

module type Fraction = sig
  (* A fraction is a rational number p/q, where q != 0.*)
  type t

  (* [make n d] is n/d. Requires d != 0. *)
  val make : int -> int -> t

  val numerator : t -> int
  val denominator : t -> int
  val to_string : t -> string
  val to_float : t -> float

  val add : t -> t -> t
  val mul : t -> t -> t
end
 *)

module type Fraction = sig
  (* A fraction is a rational number p/q, where q != 0.*)
  type t

  (* [make n d] is n/d. Requires d != 0. *)
  val make : int -> int -> t

  val numerator : t -> int
  val denominator : t -> int
  val to_string : t -> string
  val to_float : t -> float
  val add : t -> t -> t
  val mul : t -> t -> t
end

module MyFraction : Fraction = struct
    exception BadFraction

    type t = Fraction of int * int;;
    
    (* private: *)
    let rec gcd (x, y) =
        let (x, y) = if x >= y then (x, y) else (y, x) in
        if y = 0 then x else gcd(y, x mod y)
    let reduce (Fraction(n, d)) = 
        Fraction((n / gcd(n, d)), (d / gcd(n, d))) 

    (* public: *)
    let make n d = if d = 0 then raise BadFraction else
       reduce(Fraction(n, d))
    let numerator (Fraction(n, _)) = n
    let denominator (Fraction(_, d)) = d
    let to_string (Fraction(n, d)) = Printf.sprintf "%d/%d" n d
    let to_float (Fraction(n, d)) = (float_of_int n) /. (float_of_int d)
    
    let add (Fraction(n1, d1)) (Fraction(n2, d2)) = 
        reduce(Fraction(n1 * d2 + d1 * n2, d1 * d2))
    let mul (Fraction(n1, d1)) (Fraction(n2, d2)) = 
        reduce(Fraction(n1 * n2, d1 * d2))
end

open MyFraction;;

to_string (make 5 2);;
to_string (add (make 1 2) (make 6 8));;
to_string (mul (make 1 2) (make 7 3));;

make 5 0;;

module type Fraction =
  sig
    type t
    val make : int -> int -> t
    val numerator : t -> int
    val denominator : t -> int
    val to_string : t -> string
    val to_float : t -> float
    val add : t -> t -> t
    val mul : t -> t -> t
  end


module MyFraction : Fraction


- : string = "5/2"


- : string = "5/4"


- : string = "7/6"


error: runtime_error

4. For extra challenge, do binary search tree dictionary.

In [292]:
(* Exercise: binary search tree dictionary [✭✭✭]

Write a module BstDict that implements the Dictionary module type using the tree type. *)

module type Dictionary = sig
  type ('k, 'v) t

  (* The empty dictionary *)
  val empty  : ('k, 'v) t

  (* [insert k v d] produces a new dictionary [d'] with the same mappings 
   * as [d] and also a mapping from [k] to [v], even if [k] was already 
   * mapped in [d]. *)
  val insert : 'k -> 'v -> ('k,'v) t -> ('k,'v) t

  (* [lookup k d] returns the value associated with [k] in [d].  
   * raises:  [Not_found] if [k] is not mapped to any value in [d]. *)
  val lookup  : 'k -> ('k,'v) t -> 'v
end

module type Dictionary =
  sig
    type ('k, 'v) t
    val empty : ('k, 'v) t
    val insert : 'k -> 'v -> ('k, 'v) t -> ('k, 'v) t
    val lookup : 'k -> ('k, 'v) t -> 'v
  end


In [298]:
module BstDict : Dictionary = struct
    exception NotFound
    
    type 'a tree =
        Node of 'a * 'a tree * 'a tree
        | Leaf
        
    type ('k, 'v) t = ('k * 'v) tree

    let empty = Leaf
    let rec insert k v t = 
        match t with
            | Leaf -> Node((k, v), Leaf, Leaf)
            | Node((kk, vv), l, r) when k = kk -> Node((k, v), l, r)
            | Node((kk, vv), l, r) when k < kk -> Node((kk, vv), (insert k v l), r)
            | Node(tree, l, r) -> Node(tree, l, (insert k v r))
            
    let rec lookup k t =
        match t with
            | Leaf -> raise NotFound
            | Node((kk, v), _, _) when k = kk -> v
            | Node((kk, _), l, _) when k < kk -> lookup k l
            | Node(_, _, r) -> lookup k r
end

module BstDict : Dictionary


In [297]:
let make_tree l = 
    List.fold_left (fun t (k, v) -> BstDict.insert k v t) BstDict.empty l;;
        
let tree = make [("dog", 1); ("test", 0); ("ok", 6)];;
BstDict.lookup "dog" tree;;
BstDict.lookup "test" tree;;
BstDict.lookup "ok" tree;;

val make_tree : ('a * 'b) list -> ('a, 'b) BstDict.t = <fun>


val tree : (string, int) BstDict.t = <abstr>


- : int = 1


- : int = 0


- : int = 6
